# Module 4: Hourly Updates

## Purpose
This module runs hourly to monitor and adjust prices based on:
1. **UTH (Up-Till-Hour) Performance**: Cumulative qty/retailers from start of day until current hour
2. **Last Hour Performance**: Qty/retailers for the most recent hour only

## Schedule
- Runs hourly from 12 PM to 12 AM (midnight), **except** Module 3 hours (12 PM, 3 PM, 6 PM, 9 PM)
- Also runs once at 3 AM
- Active hours: 1 PM, 2 PM, 4 PM, 5 PM, 7 PM, 8 PM, 10 PM, 11 PM, 12 AM, 3 AM

## Data Flow
```
data_extraction.ipynb → Snowflake (Pricing_data_extraction)
                              ↓
                        Module 4 (this module)
                           ├── Load p80_qty, p70_retailers, std columns
                           ├── Fetch fresh: cart rules, stocks, WAC
                           ├── Calculate new_wac (from today's PRS)
                           ├── Query UTH performance (today)
                           ├── Query Last Hour performance (today)
                           ├── Query historical hour contributions
                           ├── Calculate targets & statuses (±3 std)
                           └── Generate actions (TBD)
```

## New WAC Calculation
To avoid database lag, we calculate a fresh WAC from today's purchase receipts:
- `new_wac1` = ((stocks + reflected_qty) × wac1 + unreflected_qty × item_price) / total_qty
- `new_wac_p` = wac_p × (1 + wac1_change)
- `new_wac` = new_wac_p (with fallback to current_wac)

## Status Logic (±3 Standard Deviations)
SKU status is determined by comparing actual performance to target ± 3 std:
- **Growing**: actual > target + 3×std
- **On Track**: target - 3×std ≤ actual ≤ target + 3×std
- **Dropping**: actual < target - 3×std (minimum threshold = 1)

Standard deviation columns used:
- Qty: `std_daily_240d`
- Retailers: `std_daily_retailers_240d`

## Status Outputs
- `uth_qty_status`: growing / dropping / on_track
- `uth_rets_status`: growing / dropping / on_track
- `last_hour_qty_status`: growing / dropping / on_track
- `last_hour_rets_status`: growing / dropping / on_track


In [1]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import sys
sys.path.append('..')
import setup_environment_2
# Import queries module for Snowflake access
%run queries_module.ipynb

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
CURRENT_HOUR = CAIRO_NOW.hour

print(f"Module 4: Hourly Updates")
print(f"Current Cairo Time: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current Hour: {CURRENT_HOUR}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


Queries Module | Timezone: America/Los_Angeles
✅ UTH and Last Hour functions defined

QUERIES MODULE READY

Live Data Functions:
  • get_current_stocks()
  • get_packing_units()
  • get_current_prices()
  • get_current_wac()
  • get_current_cart_rules()

UTH Performance Functions:
  • get_uth_performance()         - UTH qty/retailers (Snowflake)
  • get_hourly_distribution()     - Historical hour contributions (Snowflake)
  • get_last_hour_performance()   - Last hour qty/retailers (DWH)

Note: Market prices use MODULE_1_INPUT data
Retailer Selection Queries defined ✓
  - get_churned_dropped_retailers()
  - get_category_not_product_retailers()
  - get_out_of_cycle_retailers()
  - get_view_no_orders_retailers()
  - get_excluded_retailers()
  - get_retailers_with_quantity_discount()
  - get_retailer_main_warehouse()
Module 4: Hourly Updates
Current Cairo Time: 2026-02-09 18:01:23
Current Hour: 18


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Input/Output configuration
# Data is now loaded from Snowflake instead of Excel
INPUT_TABLE = 'MATERIALIZED_VIEWS.Pricing_data_extraction'
OUTPUT_FILE = f'module_4_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'

# Status thresholds (±3 std from target)
# - Growing: actual > target + 3*std
# - On Track: target - 3*std <= actual <= target + 3*std
# - Dropping: actual < target - 3*std (minimum threshold = 1)
STD_THRESHOLD = 3  # Number of standard deviations
MIN_DROPPING_THRESHOLD = 1  # Minimum threshold for dropping status
LOW_STOCK_DOH_THRESHOLD = 2  # SKUs with DOH <= this are protected from price reduction

# Qty growing price step limit (max 2 times per day per SKU)
MAX_QTY_GROWING_PRICE_STEPS_PER_DAY = 2

# Module 3 hours (skip these)
MODULE_3_HOURS = [12, 15, 18, 21]  # 12 PM, 3 PM, 6 PM, 9 PM

print(f"Input: {INPUT_TABLE} (today's data)")
print(f"Output: {OUTPUT_FILE}")
print(f"Status Thresholds: ±{STD_THRESHOLD} std (Dropping minimum = {MIN_DROPPING_THRESHOLD})")


Input: MATERIALIZED_VIEWS.Pricing_data_extraction (today's data)
Output: module_4_output_20260209_1801.xlsx
Status Thresholds: ±3 std (Dropping minimum = 1)


In [3]:
# =============================================================================
# LOAD PREVIOUS ACTIONS (Track qty_growing_price_step per day)
# =============================================================================
print("Loading previous hourly actions from today...")

def load_previous_hourly_actions():
    """Load previous Module 4 outputs from today to track qty_growing_price_step actions."""
    try:
        query = """
        SELECT product_id, warehouse_id, price_action
        FROM MATERIALIZED_VIEWS.pricing_hourly_push
        WHERE DATE(created_at) = CURRENT_DATE
          AND price_action = 'qty_growing_price_step'
        """
        df = query_snowflake(query)
        return df
    except Exception as e:
        print(f"  Note: Could not load previous actions: {e}")
        return pd.DataFrame()

df_previous_hourly = load_previous_hourly_actions()

# Count qty_growing_price_step actions per SKU today
if len(df_previous_hourly) > 0:
    qty_price_step_counts = (
        df_previous_hourly
        .groupby(['product_id', 'warehouse_id'])
        .size()
        .reset_index(name='qty_price_step_count')
    )
    print(f"Loaded {len(df_previous_hourly)} previous qty_growing_price_step actions")
    print(f"  Unique SKUs with price steps: {len(qty_price_step_counts)}")
else:
    qty_price_step_counts = pd.DataFrame(columns=['product_id', 'warehouse_id', 'qty_price_step_count'])
    print("No previous qty_growing_price_step actions found for today (first run or none triggered)")


Loading previous hourly actions from today...


Loaded 2 previous qty_growing_price_step actions
  Unique SKUs with price steps: 2


In [4]:
# =============================================================================
# LOAD DATA FROM SNOWFLAKE (Instead of Excel file)
# =============================================================================
print("Loading data from Snowflake...")

# Query to get today's data from Pricing_data_extraction
LOAD_QUERY = f"""
SELECT distinct * FROM {INPUT_TABLE}
WHERE created_at = '{datetime.now(CAIRO_TZ).date()}'
"""

df = query_snowflake(LOAD_QUERY)
print(f"Loaded {len(df)} records from Snowflake")

# Early exit if no data (data extraction hasn't run yet for today)
if len(df) == 0:
    print(f"\n⚠️ No data found for today ({datetime.now(CAIRO_TZ).date()})")
    print("Data extraction may not have run yet. Exiting Module 4 gracefully...")
    try:
        sys.path.insert(0, '..')
        from common_functions import send_text_slack
        send_text_slack('new-pricing-logic', f"⚠️ Module 4 skipped at {CAIRO_NOW.strftime('%H:%M')} Cairo - no data for today (data extraction not yet run)")
    except Exception as e:
        print(f"Could not send Slack notification: {e}")
    raise SystemExit("No data available - exiting gracefully")

# Ensure required columns exist with proper types
df['p80_daily_240d'] = pd.to_numeric(df.get('p80_daily_240d', 0), errors='coerce').fillna(0)
df['p70_daily_retailers_240d'] = pd.to_numeric(df.get('p70_daily_retailers_240d', 1), errors='coerce').fillna(1)
df['std_daily_240d'] = pd.to_numeric(df.get('std_daily_240d', 0), errors='coerce').fillna(0)
df['std_daily_retailers_240d'] = pd.to_numeric(df.get('std_daily_retailers_240d', 0), errors='coerce').fillna(0)
df['warehouse_id'] = df['warehouse_id'].astype(int)
df['product_id'] = df['product_id'].astype(int)
df['cohort_id'] = df['cohort_id'].astype(int) if 'cohort_id' in df.columns else None
df['commercial_min_price'] = pd.to_numeric(df.get('commercial_min_price', 0), errors='coerce')
df['commercial_min_price'] = np.round(df['commercial_min_price']*4)/4
# Get category for hourly distribution merge
if 'cat' not in df.columns and 'category' in df.columns:
    df['cat'] = df['category']

print(f"\nP80 Qty Stats: min={df['p80_daily_240d'].min():.1f}, max={df['p80_daily_240d'].max():.1f}, mean={df['p80_daily_240d'].mean():.1f}")
print(f"P70 Retailers Stats: min={df['p70_daily_retailers_240d'].min():.1f}, max={df['p70_daily_retailers_240d'].max():.1f}, mean={df['p70_daily_retailers_240d'].mean():.1f}")
print(f"Std Qty Stats: min={df['std_daily_240d'].min():.1f}, max={df['std_daily_240d'].max():.1f}, mean={df['std_daily_240d'].mean():.1f}")
print(f"Std Retailers Stats: min={df['std_daily_retailers_240d'].min():.1f}, max={df['std_daily_retailers_240d'].max():.1f}, mean={df['std_daily_retailers_240d'].mean():.1f}")

# =============================================================================
# GET FRESH DATA FROM QUERIES MODULE (Snowflake)
# =============================================================================

# 1. Current Cart Rules
df_cart_rules = get_current_cart_rules()

# Merge with main df (by cohort_id + product_id)
if 'cohort_id' in df.columns and len(df_cart_rules) > 0:
    df = df.drop(columns=['current_cart_rule'], errors='ignore')
    df = df.merge(df_cart_rules, on=['cohort_id', 'product_id'], how='left')
    df['current_cart_rule'] = df['current_cart_rule'].fillna(999)
    print(f"✅ Merged cart rules: {len(df)} records")
else:
    df['current_cart_rule'] = df.get('current_cart_rule', 999)

# 2. Current Stocks
df_stocks = get_current_stocks()

# Merge stocks (by warehouse_id + product_id)
if len(df_stocks) > 0:
    df = df.drop(columns=['stocks'], errors='ignore')
    df = df.merge(df_stocks, on=['warehouse_id', 'product_id'], how='left')
    df['stocks'] = df['stocks'].fillna(0)
    print(f"✅ Merged stocks: {len(df)} records")
else:
    df['stocks'] = df.get('stocks', 0)

# 3. Current WAC (Weighted Average Cost)
df_wac = get_current_wac()

# Merge WAC (by warehouse_id + product_id)
if len(df_wac) > 0:
    df = df.drop(columns=['wac_p'], errors='ignore')
    df = df.merge(df_wac, on=['product_id'], how='left')
    df['wac_p'] = df['wac_p'].fillna(0)
    print(f"✅ Merged WAC: {len(df)} records")
else:
    df['wac_p'] = df.get('wac_p', 0)

# 4. Current Prices
df_prices = get_current_prices()

# Merge prices (by cohort_id + product_id)
if len(df_prices) > 0:
    df = df.drop(columns=['current_price'], errors='ignore')
    df = df.merge(df_prices[['cohort_id', 'product_id', 'current_price']], on=['cohort_id', 'product_id'], how='left')
    df['current_price'] = df['current_price'].fillna(0)
    print(f"✅ Merged current prices: {len(df)} records")
else:
    df['current_price'] = df.get('current_price', 0)

print(f"\nCurrent Stock Stats: min={df['stocks'].min():.0f}, max={df['stocks'].max():.0f}, mean={df['stocks'].mean():.1f}")
print(f"Current WAC Stats: min={df['wac_p'].min():.2f}, max={df['wac_p'].max():.2f}, mean={df['wac_p'].mean():.2f}")
print(f"Current Price Stats: min={df['current_price'].min():.2f}, max={df['current_price'].max():.2f}, mean={df['current_price'].mean():.2f}")


Loading data from Snowflake...


Loaded 28076 records from Snowflake

P80 Qty Stats: min=5.0, max=1652.8, mean=13.4
P70 Retailers Stats: min=5.0, max=188.0, mean=6.1
Std Qty Stats: min=0.0, max=2574.8, mean=6.4
Std Retailers Stats: min=0.0, max=75.6, mean=1.4
Fetching current cart rules...


  Loaded 72968 records
✅ Merged cart rules: 28076 records
Fetching current stocks...


  Loaded 1865878 records


✅ Merged stocks: 28076 records
Fetching current WAC...


  Loaded 8191 records
✅ Merged WAC: 28076 records
Fetching current prices...


  Loaded 116402 records
✅ Merged current prices: 28084 records

Current Stock Stats: min=0, max=24327, mean=49.5
Current WAC Stats: min=0.01, max=2082.02, mean=228.33
Current Price Stats: min=0.01, max=2208.25, mean=238.68


In [5]:
# =============================================================================
# CALCULATE NEW WAC (Accounting for Today's Unreflected Purchases)
# =============================================================================
# This calculates a fresh WAC that accounts for today's purchase receipts
# that haven't been reflected in the database yet (to avoid lag)

print("Calculating new WAC from today's purchase receipts...")

# 1. Query today's PRS data (purchases by product_id)
prs_query = '''
WITH prs AS (
    SELECT DISTINCT 
        product_purchased_receipts.purchased_receipt_id,
        products.id AS product_id,
        product_purchased_receipts.basic_unit_count,
        product_purchased_receipts.purchased_item_count * product_purchased_receipts.basic_unit_count AS purchase_min_count,
        product_purchased_receipts.final_price / product_purchased_receipts.purchased_item_count AS final_item_price
    FROM product_purchased_receipts
    LEFT JOIN products ON products.id = product_purchased_receipts.product_id
    LEFT JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
    WHERE product_purchased_receipts.purchased_item_count <> 0
        AND purchased_receipts.purchased_receipt_status_id IN (4,5,7)
        AND purchased_receipts.date::date >= current_date
        AND product_purchased_receipts.final_price > 0 
)
SELECT 
    product_id,
    SUM(purchase_min_count) AS all_day_qty,
    AVG(final_item_price / basic_unit_count) AS item_price
FROM prs
GROUP BY 1
'''
df_prs = setup_environment_2.dwh_pg_query(prs_query, columns=['product_id', 'all_day_qty', 'item_price'])
try:
    df_prs['product_id'] = pd.to_numeric(df_prs['product_id'])
    df_prs['all_day_qty'] = pd.to_numeric(df_prs['all_day_qty'])
    df_prs['item_price'] = pd.to_numeric(df_prs['item_price'])
except:
    df_prs=pd.DataFrame(columns=['product_id', 'all_day_qty', 'item_price'])
print(f"  Loaded {len(df_prs)} PRS records (today's purchases)")

# 2. Query what's already reflected in WAC tracker
reflected_query = '''
SELECT
    t_product_id AS product_id,
    s_beg_stock AS av_stocks,
    p_purchased_item_count AS pr_qty
FROM finance.wac_tracker wt
WHERE wt.t_date::date = CURRENT_DATE
    AND p_purchased_item_count > 0 
'''
try:
    df_reflected = setup_environment_2.dwh_pg_query(reflected_query, columns=['product_id', 'av_stocks', 'pr_qty'])
    df_reflected['product_id'] = pd.to_numeric(df_reflected['product_id'])
    df_reflected['av_stocks'] = pd.to_numeric(df_reflected['av_stocks'])
    df_reflected['pr_qty'] = pd.to_numeric(df_reflected['pr_qty'])
    print(f"  Loaded {len(df_reflected)} WAC tracker records (already reflected)")
except:
    df_reflected = pd.DataFrame(columns=['product_id', 'av_stocks', 'pr_qty'])
    print("  No WAC tracker records found for today")

# 3. Query base WAC (wac1, wac_p) from all_cogs
wac_base_query = '''
SELECT 
    f.product_id,
    f.wac1,
    f.wac_p
FROM finance.all_cogs f
JOIN products ON products.id = f.product_id
JOIN categories ON products.category_id = categories.id
WHERE current_timestamp BETWEEN f.from_date AND f.to_date 
    AND NOT categories.name_ar IN (
        SELECT categories.name_ar AS cat
        FROM categories
        JOIN sections s ON s.id = categories.section_id
        WHERE categories.name_ar LIKE '%سايب%'
            OR categories.name_ar LIKE '%بالتة%'
            OR categories.section_id IN (225, 318, 285, 121, 87, 351, 417)
    )
'''
df_wac_base = query_snowflake(wac_base_query)
df_wac_base['product_id'] = pd.to_numeric(df_wac_base['product_id'])
df_wac_base['wac1'] = pd.to_numeric(df_wac_base['wac1'])
df_wac_base['wac_p'] = pd.to_numeric(df_wac_base['wac_p'])
print(f"  Loaded {len(df_wac_base)} base WAC records from all_cogs")

# 4. Merge and calculate new WAC
df_wac_calc = df_wac_base.merge(df_prs, on='product_id', how='left')
df_wac_calc = df_wac_calc.merge(df_reflected, on='product_id', how='left')
df_wac_calc = df_wac_calc.fillna(0)

# Use current_stock from main df if av_stocks is 0
df_stock_lookup = df[['product_id', 'stocks']].drop_duplicates()
df_stock_lookup = df_stock_lookup.groupby(['product_id'])['stocks'].sum().reset_index()
df_wac_calc = df_wac_calc.merge(df_stock_lookup, on='product_id', how='left')
df_wac_calc['av_stocks'] = df_wac_calc.apply(
    lambda row: row['stocks'] if row['av_stocks'] == 0 else row['av_stocks'], axis=1)

# Calculate not reflected qty (purchases not yet in WAC tracker)
df_wac_calc['not_reflected_qty'] = df_wac_calc['all_day_qty'] - df_wac_calc['pr_qty']
df_wac_calc['not_reflected_qty'] = df_wac_calc['not_reflected_qty'].clip(lower=0)  # Can't be negative

# Calculate new WAC
# new_wac1 = ((stocks + pr_qty) * wac1 + not_reflected_qty * item_price) / (stocks + pr_qty + not_reflected_qty)
denominator = df_wac_calc['av_stocks'] + df_wac_calc['pr_qty'] + df_wac_calc['not_reflected_qty']
numerator = ((df_wac_calc['av_stocks'] + df_wac_calc['pr_qty']) * df_wac_calc['wac1'] + 
             df_wac_calc['not_reflected_qty'] * df_wac_calc['item_price'])

df_wac_calc['new_wac1'] = np.where(denominator > 0, numerator / denominator, df_wac_calc['wac1'])

# Calculate wac1 change and new_wac_p
df_wac_calc['wac1_change'] = np.where(
    df_wac_calc['wac1'] > 0,
    (df_wac_calc['new_wac1'] - df_wac_calc['wac1']) / df_wac_calc['wac1'],
    0
)
df_wac_calc['new_wac_p'] = df_wac_calc['wac_p'] * (1 + df_wac_calc['wac1_change'])
df_wac_calc['new_wac_p'] = df_wac_calc['new_wac_p'].fillna(df_wac_calc['wac_p'])

# Prepare final new_wac dataframe
df_new_wac = df_wac_calc[['product_id','new_wac_p', 'not_reflected_qty']].copy()
df_new_wac=df_new_wac.drop_duplicates()

# 5. Merge new_wac into main dataframe
df = df.drop(columns=['new_wac_p','not_reflected_qty'], errors='ignore')
df = df.merge(df_new_wac, on='product_id', how='left')
df['new_wac'] = df['new_wac_p'].fillna(df['wac_p'])  # Fallback to current_wac if no new_wac

print(f"\n✅ New WAC calculated: {len(df)} records")
print(f"   Products with unreflected purchases: {(df['not_reflected_qty'] > 0).sum()}")
print(f"   New WAC Stats: min={df['new_wac'].min():.2f}, max={df['new_wac'].max():.2f}, mean={df['new_wac'].mean():.2f}")


Calculating new WAC from today's purchase receipts...


  Loaded 775 PRS records (today's purchases)
  Loaded 470 WAC tracker records (already reflected)


  Loaded 7727 base WAC records from all_cogs

✅ New WAC calculated: 28084 records
   Products with unreflected purchases: 5538
   New WAC Stats: min=0.01, max=2082.02, mean=228.33


In [6]:
# =============================================================================
# QUERY 1: TODAY'S UTH (Up-Till-Hour) PERFORMANCE
# =============================================================================
# Gets cumulative qty and retailers from start of day until current hour
# Uses get_uth_performance() from queries_module


df_uth_today = get_uth_performance()


Fetching UTH performance from Snowflake...


  Loaded 8820 UTH records


In [7]:
# =============================================================================
# QUERY 2: TODAY'S LAST HOUR PERFORMANCE (from DWH)
# =============================================================================
# Gets qty and retailers for the PREVIOUS hour only (not cumulative)
# Uses get_last_hour_performance() from queries_module (DWH/PostgreSQL)

df_last_hour = get_last_hour_performance()


Fetching last hour performance from DWH...


  Loaded 1856 last hour records from DWH


In [8]:
# =============================================================================
# QUERY 3: HISTORICAL HOURLY DISTRIBUTION (Last 4 Months) - By Category & Warehouse
# =============================================================================
# Gets:
# - avg_uth_pct_qty/retailers: Average contribution of hours 0 to (current_hour-1) to daily total
# - avg_last_hour_pct_qty/retailers: Average contribution of (current_hour-1) alone to daily total
# Uses get_hourly_distribution() from queries_module

df_hourly_dist = get_hourly_distribution()
print(f"\nAvg UTH % (qty): {df_hourly_dist['avg_uth_pct_qty'].mean()*100:.1f}%")
print(f"Avg Last Hour % (qty): {df_hourly_dist['avg_last_hour_pct_qty'].mean()*100:.1f}%")


Fetching hourly distribution from Snowflake...


  Loaded 771 hourly distribution records

Avg UTH % (qty): 61.7%
Avg Last Hour % (qty): 6.3%


In [9]:
# =============================================================================
# MERGE DATA
# =============================================================================
print("Merging performance data with base data...")

# Merge UTH today data
if len(df_uth_today) > 0:
    df = df.merge(df_uth_today, on=['warehouse_id', 'product_id'], how='left')
else:
    df['uth_qty'] = 0
    df['uth_nmv'] = 0
    df['uth_retailers'] = 0

# Merge last hour data
if len(df_last_hour) > 0:
    df = df.merge(df_last_hour, on=['warehouse_id', 'product_id'], how='left')
else:
    df['last_hour_qty'] = 0
    df['last_hour_nmv'] = 0
    df['last_hour_retailers'] = 0

# Merge hourly distribution (by warehouse_id + cat)
if len(df_hourly_dist) > 0:
    df = df.merge(df_hourly_dist, on=['warehouse_id', 'cat'], how='left')
else:
    df['avg_uth_pct_qty'] = 0.5
    df['avg_uth_pct_retailers'] = 0.5
    df['avg_last_hour_pct_qty'] = 0.05
    df['avg_last_hour_pct_retailers'] = 0.05

# Fill NaN values
df['uth_qty'] = df['uth_qty'].fillna(0)
df['uth_nmv'] = df['uth_nmv'].fillna(0)
df['uth_retailers'] = df['uth_retailers'].fillna(0)
df['last_hour_qty'] = df['last_hour_qty'].fillna(0)
df['last_hour_nmv'] = df['last_hour_nmv'].fillna(0)
df['last_hour_retailers'] = df['last_hour_retailers'].fillna(0)
df['avg_uth_pct_qty'] = df['avg_uth_pct_qty'].fillna(0.5)
df['avg_uth_pct_retailers'] = df['avg_uth_pct_retailers'].fillna(0.5)
df['avg_last_hour_pct_qty'] = df['avg_last_hour_pct_qty'].fillna(0.05)
df['avg_last_hour_pct_retailers'] = df['avg_last_hour_pct_retailers'].fillna(0.05)

print(f"✅ Merged data: {len(df)} records")
print(f"\nUTH Qty Stats: min={df['uth_qty'].min():.0f}, max={df['uth_qty'].max():.0f}, mean={df['uth_qty'].mean():.1f}")
print(f"Last Hour Qty Stats: min={df['last_hour_qty'].min():.0f}, max={df['last_hour_qty'].max():.0f}, mean={df['last_hour_qty'].mean():.1f}")
print(f"Current Cart Rule Stats: min={df['current_cart_rule'].min():.0f}, max={df['current_cart_rule'].max():.0f}, mean={df['current_cart_rule'].mean():.1f}")


Merging performance data with base data...
✅ Merged data: 28084 records

UTH Qty Stats: min=0, max=426, mean=2.3
Last Hour Qty Stats: min=0, max=266, mean=0.2
Current Cart Rule Stats: min=2, max=677, mean=19.6


In [10]:
# =============================================================================
# CALCULATE TARGETS AND STATUSES
# =============================================================================
print("Calculating UTH and Last Hour targets and statuses...")

def get_status_std(actual, target, std, is_qty=True):
    """
    Determine status based on ±3 std from target.
    - Growing: actual > target + 3*std
    - On Track: target - 3*std <= actual <= target + 3*std
    - Dropping: actual < target - 3*std (minimum threshold = 1)
    
    Parameters:
    - actual: actual performance value
    - target: target value (p80 for qty, p70 for retailers)
    - std: standard deviation (std_daily_240d or std_daily_retailers_240d)
    - is_qty: True for qty metrics, False for retailer metrics
    """
    upper_bound = target + STD_THRESHOLD * std
    lower_bound = max(target - STD_THRESHOLD * std, MIN_DROPPING_THRESHOLD)
    
    if actual > upper_bound:
        return 'growing'
    elif actual < lower_bound:
        return 'dropping'
    else:
        return 'on_track'

# Calculate UTH targets
# UTH target = p80_qty * avg_uth_pct (historical % of day that should be done by now)
df['uth_qty_target'] = df['p80_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate UTH std thresholds (scaled by UTH percentage)
df['uth_qty_std'] = df['std_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_std'] = df['std_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate Last Hour targets
# Last hour target = p80_qty * avg_last_hour_pct (historical % of day for this hour)
df['last_hour_qty_target'] = df['p80_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate Last Hour std thresholds (scaled by last hour percentage)
df['last_hour_qty_std'] = df['std_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_std'] = df['std_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate statuses using ±3 std thresholds
df['uth_qty_status'] = df.apply(
    lambda row: get_status_std(row['uth_qty'], row['uth_qty_target'], row['uth_qty_std'], is_qty=True), axis=1)
df['uth_rets_status'] = df.apply(
    lambda row: get_status_std(row['uth_retailers'], row['uth_rets_target'], row['uth_rets_std'], is_qty=False), axis=1)
df['last_hour_qty_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_qty'], row['last_hour_qty_target'], row['last_hour_qty_std'], is_qty=True), axis=1)
df['last_hour_rets_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_retailers'], row['last_hour_rets_target'], row['last_hour_rets_std'], is_qty=False), axis=1)

print(f"✅ Targets and statuses calculated using ±{STD_THRESHOLD} std thresholds")

# Summary
print(f"\n{'='*60}")
print("UTH STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nUTH Qty Status:")
print(df['uth_qty_status'].value_counts().to_string())
print(f"\nUTH Retailers Status:")
print(df['uth_rets_status'].value_counts().to_string())

print(f"\n{'='*60}")
print("LAST HOUR STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nLast Hour Qty Status:")
print(df['last_hour_qty_status'].value_counts().to_string())
print(f"\nLast Hour Retailers Status:")
print(df['last_hour_rets_status'].value_counts().to_string())


Calculating UTH and Last Hour targets and statuses...


✅ Targets and statuses calculated using ±3 std thresholds

UTH STATUS DISTRIBUTION

UTH Qty Status:
uth_qty_status
dropping    20707
on_track     7123
growing       254

UTH Retailers Status:
uth_rets_status
dropping    22239
on_track     5766
growing        79

LAST HOUR STATUS DISTRIBUTION

Last Hour Qty Status:
last_hour_qty_status
dropping    26368
on_track     1093
growing       623

Last Hour Retailers Status:
last_hour_rets_status
dropping    26364
growing       966
on_track      754


In [11]:
df[df['product_id']==589]#[['region','uth_qty_status','new_price']]

,cohort_id,product_id,region,warehouse_id,warehouse,sku,brand,cat,wac1,current_margin,...,uth_qty_std,uth_rets_std,last_hour_qty_target,last_hour_rets_target,last_hour_qty_std,last_hour_rets_std,uth_qty_status,uth_rets_status,last_hour_qty_status,last_hour_rets_status
6465,703,589,Delta West,337,El-Mahala,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.025410,...,30.434563,7.171833,7.717478,2.384594,3.527242,0.830224,on_track,on_track,on_track,on_track
9114,704,589,Delta East,339,Mansoura FC,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.017658,...,31.854405,5.029363,5.324778,1.625572,3.844982,0.607057,on_track,on_track,dropping,dropping
11644,1126,589,Upper Egypt,401,Bani sweif,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.060368,...,20.167253,3.830228,4.334004,1.049046,2.203889,0.444723,on_track,on_track,dropping,dropping
13427,700,589,Cairo,1,Mostorod,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.060368,...,127.900925,15.796643,21.707665,4.739356,12.911320,1.643594,on_track,on_track,dropping,dropping
16181,704,589,Delta East,170,Sharqya,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.017658,...,34.562817,5.788727,7.105854,1.759848,3.744347,0.630888,on_track,on_track,on_track,on_track
18055,1124,589,Upper Egypt,501,Assiut FC,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,-0.029120,...,11.271547,2.703162,2.526116,0.646724,1.327593,0.317826,on_track,on_track,dropping,dropping
19181,1125,589,Upper Egypt,632,Sohag,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.068469,...,8.976329,2.417376,2.240439,0.677727,1.020135,0.269439,dropping,dropping,dropping,dropping
19939,701,589,Giza,236,Barageel,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.067464,...,50.653572,8.815642,13.603486,3.106257,4.863648,0.895331,on_track,dropping,on_track,on_track
19940,701,589,Giza,962,Sakkarah,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,0.067464,...,47.697007,8.928157,12.180660,2.711504,4.576000,0.872904,on_track,on_track,on_track,on_track
22516,702,589,Alexandria,797,Khorshed Alex,بيبسى - 2.43 لتر,بيبسي,حاجه ساقعه,220.525805,-0.029120,...,40.267816,5.668010,7.368791,1.600739,4.873237,0.624509,growing,on_track,dropping,dropping


In [12]:
# =============================================================================
# FETCH MARKET DATA AND MARGIN TIERS
# =============================================================================
print("Fetching market data and margin tiers...")

# Import market_data_module for get_market_data()
%run market_data_module.ipynb

# 1. Get margin tiers (already available from queries_module)
df_margin_tiers = get_margin_tiers()
print(f"✅ Loaded {len(df_margin_tiers)} margin tier records")

# 2. Get market data
df_market = get_market_data()
print(f"✅ Loaded {len(df_market)} market data records")

# Verify cohort_id is available in all dataframes
print(f"\nChecking cohort_id availability:")
print(f"  - df has cohort_id: {'cohort_id' in df.columns}")
print(f"  - df_margin_tiers has cohort_id: {'cohort_id' in df_margin_tiers.columns}")
print(f"  - df_market has cohort_id: {'cohort_id' in df_market.columns}")

# 3. Merge margin tiers with df (by cohort_id + product_id) - ALL COLUMNS
merge_keys = ['cohort_id', 'product_id']
df = df.drop(columns=[c for c in df_margin_tiers.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_margin_tiers, on=merge_keys, how='left')
print(f"\n✅ Merged margin tiers: {len(df)} records")
print(f"   Margin tier columns added: {[c for c in df_margin_tiers.columns if c not in merge_keys]}")

# 4. Merge market data with df (by cohort_id + product_id) - ALL COLUMNS
df = df.drop(columns=[c for c in df_market.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_market, on=merge_keys, how='left')
print(f"✅ Merged market data: {len(df)} records")
print(f"   Market columns added: {[c for c in df_market.columns if c not in merge_keys]}")

# 5. Calculate current margin based on current_price and new_wac
df['current_margin'] = (df['current_price'] - df['new_wac']) / df['current_price']
df['current_margin'] = df['current_margin'].fillna(0)

print(f"\nMargin Stats: min={df['current_margin'].min():.2%}, max={df['current_margin'].max():.2%}, mean={df['current_margin'].mean():.2%}")

# =============================================================================
# FETCH HIGH DOH SKUS FROM MODULE 3 OUTPUT
# =============================================================================
# SKUs with high DOH should NOT have price increases (only price decreases in Module 3)
print("Fetching high DOH SKUs from Module 3...")

HIGH_DOH_QUERY = '''
SELECT DISTINCT product_id, warehouse_id, 1 as is_high_doh
FROM MATERIALIZED_VIEWS.pricing_periodic_push
WHERE created_at::DATE = CURRENT_DATE
  AND (uth_status = 'High DOH' OR price_action LIKE '%doh%')
'''
try:
    df_high_doh = query_snowflake(HIGH_DOH_QUERY)
    print(f"  Loaded {len(df_high_doh)} high DOH SKUs from Module 3")
except:
    df_high_doh = pd.DataFrame(columns=['product_id', 'warehouse_id', 'is_high_doh'])
    print("  No high DOH data found (Module 3 may not have run yet)")

# Merge high DOH flag
df['is_high_doh'] = 0  # Initialize column first
if len(df_high_doh) > 0:
    df = df.drop(columns=['is_high_doh'], errors='ignore')
    df = df.merge(df_high_doh[['product_id', 'warehouse_id', 'is_high_doh']], 
                  on=['product_id', 'warehouse_id'], how='left')
    df['is_high_doh'] = df['is_high_doh'].fillna(0).astype(int)
print(f"  SKUs marked as high DOH: {(df['is_high_doh'] == 1).sum()}")

# =============================================================================
# FILTER SKUS FOR ACTION
# =============================================================================
# Filter SKUs that need action based on:
# Pre-condition: below_min_stock_flag != 1 (must be sellable)
# Condition A: new_wac > wac_p * 1.005 (WAC increased by more than 0.5%)
# Condition B: Last hour growing (qty OR rets) AND UTH (one growing + other on_track/growing) AND last_hour_qty > 5 AND NOT high DOH

print("Filtering SKUs for action...")

# Skip SKUs with below_min_stock_flag (stock < min selling unit qty)
can_sell = df['below_min_stock_flag'].fillna(0) != 1

# Condition A: WAC increased significantly
condition_a = (df['new_wac'] > df['wac_p'] * 1.005)

# Condition B: Last hour growing AND UTH performing well AND last_hour_qty > 5 AND NOT high DOH
# UTH good = one must be 'growing' AND the other must be 'on_track' or 'growing'
# High DOH SKUs should NOT get price increases (Module 3 handles their price reductions)
last_hour_growing = (df['last_hour_qty_status'] == 'growing') | (df['last_hour_rets_status'] == 'growing')
uth_good = (
    ((df['uth_qty_status'] == 'growing') & (df['uth_rets_status'].isin(['on_track', 'growing']))) |
    ((df['uth_rets_status'] == 'growing') & (df['uth_qty_status'].isin(['on_track', 'growing'])))
)
last_hour_qty_threshold = df['last_hour_qty'] > 5
not_high_doh = df['is_high_doh'] == 0
condition_b = last_hour_growing & uth_good & last_hour_qty_threshold & can_sell #& not_high_doh

# Condition C: Current price below commercial minimum (must enforce commercial min)
condition_c = (
    (df['commercial_min_price'].notna()) & 
    (df['commercial_min_price'] > 0) & 
    (df['current_price'] < df['commercial_min_price'])
)

# Combine conditions (A OR B OR C)
df['needs_action'] = condition_a | condition_b | condition_c
df['action_reason'] = 'none'
df.loc[condition_a & ~condition_b & ~condition_c, 'action_reason'] = 'wac_increase'
df.loc[~condition_a & condition_b & ~condition_c, 'action_reason'] = 'growing_performance'
df.loc[~condition_a & ~condition_b & condition_c, 'action_reason'] = 'below_commercial_min'
df.loc[condition_a & condition_b & ~condition_c, 'action_reason'] = 'both'
df.loc[condition_a & ~condition_b & condition_c, 'action_reason'] = 'wac_and_commercial'
df.loc[~condition_a & condition_b & condition_c, 'action_reason'] = 'growing_and_commercial'
df.loc[condition_a & condition_b & condition_c, 'action_reason'] = 'all'

# Define important columns for df_action
action_columns = [
    # Identification
    'warehouse_id', 'product_id', 'cohort_id', 'sku', 'region', 'brand', 'cat',
    # Price & Cost
    'current_price', 'wac_p', 'new_wac', 'current_margin', 'commercial_min_price',
    # Inventory & Rules
    'stocks', 'current_cart_rule', 'doh',
    # Performance Targets
    'p80_daily_240d', 'p70_daily_retailers_240d', 'std_daily_240d', 'std_daily_retailers_240d',
    # Refill columns for cart rule calculation
    'normal_refill', 'refill_stddev', 'target_margin',
    # UTH Status
    'uth_qty', 'uth_qty_status', 'uth_retailers', 'uth_rets_status',
    # Last Hour Status
    'last_hour_qty', 'last_hour_qty_status', 'last_hour_retailers', 'last_hour_rets_status',
    # Market Margins
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market',
    # Margin Tiers (excluding: effective_min_margin, max_boundary, margin_step, margin_tier_below)
    'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
    'margin_tier_above_1', 'margin_tier_above_2',
    # Action Flags
    'needs_action', 'action_reason'
]

# Filter to only columns that exist in df
action_columns = [c for c in action_columns if c in df.columns]

# Filter to action SKUs with selected columns only
df_action = df[df['needs_action']][action_columns].copy()

# Merge previous qty_growing_price_step counts
df_action = df_action.merge(qty_price_step_counts, on=['product_id', 'warehouse_id'], how='left')
df_action['qty_price_step_count'] = df_action['qty_price_step_count'].fillna(0).astype(int)

print(f"\n{'='*60}")
print("SKUs FILTERED FOR ACTION")
print(f"{'='*60}")
print(f"\nTotal SKUs needing action: {len(df_action)} out of {len(df)} ({len(df_action)/len(df)*100:.1f}%)")
print(f"\nBreakdown by reason:")
print(df_action['action_reason'].value_counts().to_string())

print(f"\n--- Condition A (WAC increase > 0.5%): {condition_a.sum()} SKUs")
print(f"--- Condition B (Last hour growing + UTH good + qty > 5 + NOT high DOH): {condition_b.sum()} SKUs")
print(f"--- Condition C (Below commercial minimum): {condition_c.sum()} SKUs")
print(f"--- High DOH SKUs blocked from price increase: {(df['is_high_doh'] == 1).sum()}")

# Show sample
print(f"\n{'='*60}")
print("SAMPLE ACTION SKUs")
print(f"{'='*60}")
print(f"\nTotal columns in df_action: {len(df_action.columns)}")
print(f"Columns: {df_action.columns.tolist()}")
display(df_action.head(10))


Fetching market data and margin tiers...


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


Market Data Module loaded at 2026-02-09 18:03:17 Cairo time
Snowflake timezone: America/Los_Angeles
All queries defined ✓
Helper functions defined ✓
get_market_data() function defined ✓
get_margin_tiers() function defined ✓

MARKET DATA MODULE READY

Available functions (NO INPUT REQUIRED):
  - get_market_data()   : Fetch and process all market prices
  - get_margin_tiers()  : Fetch and calculate margin tiers

Usage:
  %run market_data_module.ipynb
  df_market = get_market_data()
  df_tiers = get_margin_tiers()

FETCHING MARGIN TIERS
Timestamp: 2026-02-09 18:03:18 Cairo time

Step 1: Fetching margin boundaries from PRODUCT_STATISTICS...


    Loaded 18047 records

Step 2: Adding cohort IDs...
    Records with cohorts: 24890

Step 3: Calculating margin tiers...

MARGIN TIERS COMPLETE
Total records: 24890

Margin Tier Structure:
  margin_tier_below:   effective_min - step (1 below)
  margin_tier_1:       effective_min_margin
  margin_tier_2:       effective_min + 1*step
  margin_tier_3:       effective_min + 2*step
  margin_tier_4:       effective_min + 3*step
  margin_tier_5:       max_boundary
  margin_tier_above_1: max_boundary + 1*step
  margin_tier_above_2: max_boundary + 2*step
✅ Loaded 24890 margin tier records

FETCHING MARKET DATA
Timestamp: 2026-02-09 18:03:20 Cairo time

Step 1: Fetching raw price data...
  1.1 Ben Soliman prices...


      Loaded 1539 records
  1.2 Marketplace prices...


...

/tmp/ipykernel_18980/3245917641.py:139: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


    Records after group processing: 24722

Step 5: Adding WAC and margin data...
    Records with WAC: 24324

Step 6: Filtering by price coverage...
    Records after price coverage filter: 13071

Step 7: Calculating price percentiles...


    Records after price analysis: 12429

Step 8: Converting prices to margins...

MARKET DATA COMPLETE
Total records: 12429
  - With marketplace prices: 11744
  - With scrapped prices: 5834
  - With Ben Soliman prices: 8255
✅ Loaded 12429 market data records

Checking cohort_id availability:
  - df has cohort_id: True
  - df_margin_tiers has cohort_id: True
  - df_market has cohort_id: True

✅ Merged margin tiers: 28084 records
   Margin tier columns added: ['region', 'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm', 'effective_min_margin', 'margin_step', 'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']
✅ Merged market data: 28084 records
   Market columns added: ['region', 'ben_soliman_price', 'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max', 'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped', 'minimum', 'perce

  Loaded 248 high DOH SKUs from Module 3
  SKUs marked as high DOH: 248
Filtering SKUs for action...

SKUs FILTERED FOR ACTION

Total SKUs needing action: 91 out of 28084 (0.3%)

Breakdown by reason:
action_reason
wac_increase           81
growing_performance    10

--- Condition A (WAC increase > 0.5%): 81 SKUs
--- Condition B (Last hour growing + UTH good + qty > 5 + NOT high DOH): 10 SKUs
--- Condition C (Below commercial minimum): 0 SKUs
--- High DOH SKUs blocked from price increase: 248

SAMPLE ACTION SKUs

Total columns in df_action: 47
Columns: ['warehouse_id', 'product_id', 'cohort_id', 'sku', 'region', 'brand', 'cat', 'current_price', 'wac_p', 'new_wac', 'current_margin', 'commercial_min_price', 'stocks', 'current_cart_rule', 'doh', 'p80_daily_240d', 'p70_daily_retailers_240d', 'std_daily_240d', 'std_daily_retailers_240d', 'normal_refill', 'refill_stddev', 'target_margin', 'uth_qty', 'uth_qty_status', 'uth_retailers', 'uth_rets_status', 'last_hour_qty', 'last_hour_qty_status',

,warehouse_id,product_id,cohort_id,sku,region,brand,cat,current_price,wac_p,new_wac,...,margin_tier_1,margin_tier_2,margin_tier_3,margin_tier_4,margin_tier_5,margin_tier_above_1,margin_tier_above_2,needs_action,action_reason,qty_price_step_count
0,501,2057,1124,كابتشينو كوفى بريك كلاسيك 8 ظرف - 18.5 جم,Upper Egypt,كوفي بريك,قهوة,62.50,59.901833,60.335653,...,0.029421,0.035816,0.042211,0.048605,0.055000,0.061395,0.067789,True,wac_increase,0
1,170,2263,704,ريحانة لوبيا بلدي- 500 جم,NaN,ريحانة,بقوليات,70.50,68.362058,69.262870,...,0.059908,0.066108,0.072308,0.078508,0.084708,0.090908,0.097108,True,wac_increase,0
2,236,2263,701,ريحانة لوبيا بلدي- 500 جم,NaN,ريحانة,بقوليات,70.75,68.362058,69.262870,...,0.048796,0.062737,0.076679,0.090621,0.104562,0.118504,0.132445,True,wac_increase,0
3,797,2055,702,كابتشينو كوفى بريك بندق 8 ظرف - 18.5 جم,Alexandria,كوفي بريك,قهوة,61.50,59.043438,59.953545,...,0.028000,0.034750,0.041500,0.048250,0.055000,0.061750,0.068500,True,wac_increase,0
4,797,2775,702,سبرايت - 2.45 لتر,Alexandria,كوكا كولا,حاجه ساقعه,222.50,204.680292,205.823809,...,0.056000,0.067033,0.078066,0.089098,0.100131,0.111164,0.122197,True,wac_increase,0
5,962,12212,701,مناديل سمايل سحب 550 منديل,NaN,سمايل,ورقيات,247.50,247.151221,252.749059,...,0.040617,0.049609,0.058601,0.067594,0.076586,0.085578,0.094571,True,wac_increase,0
6,501,2775,1124,سبرايت - 2.45 لتر,Upper Egypt,كوكا كولا,حاجه ساقعه,213.00,204.680292,205.823809,...,0.056000,0.070168,0.084337,0.098505,0.112674,0.126842,0.141010,True,wac_increase,0
7,339,12212,704,مناديل سمايل سحب 550 منديل,NaN,سمايل,ورقيات,255.50,247.151221,252.749059,...,0.040617,0.048719,0.056820,0.064922,0.073024,0.081126,0.089228,True,wac_increase,0
8,962,12924,701,زيت ثمرات خليط - 700 مل,NaN,ثمرات,زيوت,575.25,530.395094,530.395094,...,0.035000,0.043590,0.052179,0.060769,0.069358,0.077948,0.086537,True,growing_performance,0
9,337,2775,703,سبرايت - 2.45 لتر,Delta West,كوكا كولا,حاجه ساقعه,216.75,204.680292,205.823809,...,0.056000,0.067028,0.078057,0.089085,0.100113,0.111142,0.122170,True,wac_increase,0


In [13]:
# =============================================================================
# ACTION LOGIC
# =============================================================================
print("Calculating actions for filtered SKUs...")

# Step 0: Preparation - Calculate normal_refill_3std
df_action['normal_refill_3std'] = df_action['normal_refill'] + 3 * df_action['refill_stddev']

# Initialize output columns
df_action['new_price'] = np.nan
df_action['new_cart_rule'] = np.nan
df_action['price_action'] = 'none'
df_action['cart_rule_action'] = 'none'

# =============================================================================
# HELPER: Calculate all price tiers from margins using wac_p
# =============================================================================
def calculate_price_from_margin(wac, margin):
    """Calculate price from WAC and margin: price = wac / (1 - margin)"""
    if pd.isna(margin) or margin >= 1:
        return np.nan
    return wac / (1 - margin)

# Calculate market prices (using wac_p)
market_margin_cols = ['market_min', 'market_25', 'market_50', 'market_75', 'market_max']
for col in market_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# Calculate margin tier prices (using wac_p)
tier_margin_cols = ['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
                    'margin_tier_above_1', 'margin_tier_above_2']
for col in tier_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# =============================================================================
# HELPER: Find first price above threshold
# =============================================================================
def find_first_price_above(row, threshold, price_cols):
    """Find the first price in price_cols that is above threshold."""
    for col in price_cols:
        price = row.get(col, np.nan)
        if pd.notna(price) and price > threshold:
            return price
    return np.nan

# Define price column order (market first, then tiers)
market_price_cols = ['market_min_price', 'market_25_price', 'market_50_price', 'market_75_price', 'market_max_price']
tier_price_cols = ['margin_tier_1_price', 'margin_tier_2_price', 'margin_tier_3_price', 
                   'margin_tier_4_price', 'margin_tier_5_price', 
                   'margin_tier_above_1_price', 'margin_tier_above_2_price']
all_price_cols = market_price_cols + tier_price_cols

# =============================================================================
# STEP 1: WAC Increase Actions (action_reason = 'both' or 'wac_increase')
# =============================================================================
# Only apply if: current_price < new_wac AND current_price > wac_p
# (price is squeezed between old and new WAC)
wac_increase_mask = (
    df_action['action_reason'].isin(['both', 'wac_increase']) &
    (df_action['current_price'] < df_action['new_wac']) &
    (df_action['current_price'] > df_action['wac_p'])
)

def get_wac_increase_price(row):
    """Find first market/tier price above current_price."""
    # Find first price above current_price (not new_wac)
    new_price = find_first_price_above(row, row['current_price'], all_price_cols)
    return new_price

df_action.loc[wac_increase_mask, 'new_price'] = df_action[wac_increase_mask].apply(get_wac_increase_price, axis=1)
df_action.loc[wac_increase_mask, 'price_action'] = 'wac_increase'

print(f"  WAC increase actions: {wac_increase_mask.sum()} SKUs")
print(f"    (SKUs where wac_p < current_price < new_wac)")

# =============================================================================
# STEP 2: Growing Performance Actions (action_reason = 'growing_performance')
# =============================================================================
growing_mask = df_action['action_reason'] == 'growing_performance'

# Case A: Retailers growing (with or without qty)
rets_growing_mask = growing_mask & (df_action['last_hour_rets_status'] == 'growing')

def get_rets_growing_price(row):
    """Find price for rets growing: market/tier >= current_price * 1.005, or fallback to target margin."""
    min_price = row['current_price'] * 1.005
    
    # Try market prices first
    new_price = find_first_price_above(row, min_price, market_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Try tier prices
    new_price = find_first_price_above(row, min_price, tier_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Fallback: wac_p / (1 - (target_margin * 1.15))
    target_margin = row.get('target_margin', 0)
    if pd.notna(target_margin) and target_margin * 1.15 < 1:
        fallback_price = row['wac_p'] / (1 - (target_margin * 1.15))
        # If fallback price is lower than current price, use current_price * 1.01
        if fallback_price < row['current_price']:
            return row['current_price'] * 1.01
        return fallback_price
    
    return np.nan

df_action.loc[rets_growing_mask, 'new_price'] = df_action[rets_growing_mask].apply(get_rets_growing_price, axis=1)
df_action.loc[rets_growing_mask, 'price_action'] = 'rets_growing'

print(f"  Rets growing actions (price increase): {rets_growing_mask.sum()} SKUs")

# Case B: Qty growing only (rets NOT growing)
qty_only_growing_mask = growing_mask & (df_action['last_hour_qty_status'] == 'growing') & (df_action['last_hour_rets_status'] != 'growing')

def get_qty_growing_cart_rule(row):
    """Calculate new cart rule for qty growing only."""
    # Calculate qty per retailer
    uth_retailers = row.get('uth_retailers', 0)
    if uth_retailers == 0:
        qty_per_retailer = np.inf
    else:
        qty_per_retailer = row['uth_qty'] / uth_retailers
    
    # Get the three options
    option1 = row.get('normal_refill_3std', np.inf)
    option2 = qty_per_retailer
    option3 = row.get('current_cart_rule', np.inf) * 0.85
    
    # Return minimum of the three (excluding inf/nan)
    options = [opt for opt in [option1, option2, option3] if pd.notna(opt) and opt != np.inf]
    if options:
        return min(options)
    return np.nan

df_action.loc[qty_only_growing_mask, 'new_cart_rule'] = df_action[qty_only_growing_mask].apply(get_qty_growing_cart_rule, axis=1)

# Round and apply min/max constraints to new_cart_rule
df_action['new_cart_rule'] = df_action['new_cart_rule'].round()
df_action['new_cart_rule'] = df_action['new_cart_rule'].clip(lower=5, upper=150)

# LOW STOCK PROTECTION: Cap cart rule at normal_refill for low stock SKUs with demand
# This prevents cart expansion that could deplete inventory before next receiving
if 'doh' in df_action.columns:
    low_stock_mask = (
        (df_action['doh'].fillna(999) <= LOW_STOCK_DOH_THRESHOLD) & 
        (df_action['uth_qty'].fillna(0) > 0) &
        (df_action['new_cart_rule'].notna())
    )
    if low_stock_mask.sum() > 0:
        # Cap cart rule at normal_refill for low stock SKUs
        df_action.loc[low_stock_mask, 'new_cart_rule'] = df_action.loc[low_stock_mask].apply(
            lambda row: min(row['new_cart_rule'], np.ceil(row.get('normal_refill', row['new_cart_rule']))), axis=1
        )
        print(f"  Low stock protection: {low_stock_mask.sum()} SKUs had cart rule capped at normal_refill")

df_action.loc[qty_only_growing_mask, 'cart_rule_action'] = 'qty_growing'

print(f"  Qty growing actions (cart rule change): {qty_only_growing_mask.sum()} SKUs")
print(f"    (Cart rule: rounded, min=5, max=150)")

# =============================================================================
# NEW RULE: If new_cart_rule <= normal_refill AND < 2 price steps today AND DOH <= 30
# =============================================================================
HIGH_DOH_THRESHOLD = 30
can_do_price_step = df_action['qty_price_step_count'] < MAX_QTY_GROWING_PRICE_STEPS_PER_DAY
not_high_doh = df_action['doh'].fillna(0) <= HIGH_DOH_THRESHOLD
qty_growing_needs_price = qty_only_growing_mask & (df_action['new_cart_rule'] <= df_action['normal_refill']) & can_do_price_step & not_high_doh

def get_qty_growing_price_step(row):
    """Find next tier price above current price (1 step increase)."""
    return find_first_price_above(row, row['current_price'] * 1.001, tier_price_cols)

df_action.loc[qty_growing_needs_price, 'new_price'] = df_action[qty_growing_needs_price].apply(get_qty_growing_price_step, axis=1)
df_action.loc[qty_growing_needs_price, 'price_action'] = 'qty_growing_price_step'

qty_growing_blocked_limit = qty_only_growing_mask & (df_action['new_cart_rule'] <= df_action['normal_refill']) & ~can_do_price_step
qty_growing_blocked_doh = qty_only_growing_mask & (df_action['new_cart_rule'] <= df_action['normal_refill']) & can_do_price_step & ~not_high_doh
print(f"  Qty growing with price step: {qty_growing_needs_price.sum()} SKUs")
print(f"  Qty growing blocked (already {MAX_QTY_GROWING_PRICE_STEPS_PER_DAY}x today): {qty_growing_blocked_limit.sum()} SKUs")
print(f"  Qty growing blocked (high DOH > {HIGH_DOH_THRESHOLD}): {qty_growing_blocked_doh.sum()} SKUs")

# =============================================================================
# FINAL STEP: ENFORCE COMMERCIAL MINIMUM PRICE
# =============================================================================
# For all SKUs in df_action, ensure new_price >= commercial_min_price
print(f"\n{'='*60}")
print("ENFORCING COMMERCIAL MINIMUM PRICE")
print(f"{'='*60}")

has_commercial_min = (df_action['commercial_min_price'].notna()) & (df_action['commercial_min_price'] > 0)

# For SKUs with new_price already calculated (growing/wac): MAX(new_price, commercial_min)
has_new_price = df_action['new_price'].notna()
below_min_with_new = has_commercial_min & has_new_price & (df_action['new_price'] < df_action['commercial_min_price'])
df_action.loc[below_min_with_new, 'new_price'] = df_action.loc[below_min_with_new, 'commercial_min_price']
df_action.loc[below_min_with_new, 'price_action'] = df_action.loc[below_min_with_new, 'price_action'].astype(str) + '_commercial_enforced'

# For SKUs without new_price (only commercial min violation): new_price = commercial_min
no_new_price = df_action['new_price'].isna()
below_min_only = has_commercial_min & no_new_price & (df_action['current_price'] < df_action['commercial_min_price'])
df_action.loc[below_min_only, 'new_price'] = df_action.loc[below_min_only, 'commercial_min_price']
df_action.loc[below_min_only, 'price_action'] = 'commercial_min_enforced'

print(f"  SKUs with commercial min: {has_commercial_min.sum()}")
print(f"  Growing SKUs with commercial min applied: {below_min_with_new.sum()}")
print(f"  SKUs only needing commercial min: {below_min_only.sum()}")
print(f"  Total commercial min enforcements: {below_min_with_new.sum() + below_min_only.sum()}")

# =============================================================================
# SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("ACTION SUMMARY")
print(f"{'='*60}")
print(f"\nPrice Actions:")
print(df_action['price_action'].value_counts().to_string())
print(f"\nCart Rule Actions:")
print(df_action['cart_rule_action'].value_counts().to_string())

# Show SKUs with new prices
price_changes = df_action[df_action['new_price'].notna()]
print(f"\nSKUs with new price: {len(price_changes)}")
if len(price_changes) > 0:
    print(f"  Avg price change: {((price_changes['new_price'] - price_changes['current_price']) / price_changes['current_price']).mean()*100:.1f}%")

# Show SKUs with new cart rules
cart_changes = df_action[df_action['new_cart_rule'].notna()]
print(f"\nSKUs with new cart rule: {len(cart_changes)}")
if len(cart_changes) > 0:
    print(f"  Avg cart rule change: {((cart_changes['new_cart_rule'] - cart_changes['current_cart_rule']) / cart_changes['current_cart_rule']).mean()*100:.1f}%")

# Display sample
print(f"\n{'='*60}")
print("SAMPLE OUTPUT")
print(f"{'='*60}")
output_cols = ['sku', 'current_price', 'new_price', 'current_cart_rule', 'new_cart_rule', 
               'price_action', 'cart_rule_action', 'action_reason']
output_cols = [c for c in output_cols if c in df_action.columns]
display(df_action[df_action['new_price'].notna() | df_action['new_cart_rule'].notna()][output_cols].head(15))


Calculating actions for filtered SKUs...
  WAC increase actions: 3 SKUs
    (SKUs where wac_p < current_price < new_wac)
  Rets growing actions (price increase): 8 SKUs
  Low stock protection: 1 SKUs had cart rule capped at normal_refill
  Qty growing actions (cart rule change): 2 SKUs
    (Cart rule: rounded, min=5, max=150)
  Qty growing with price step: 1 SKUs
  Qty growing blocked (already 2x today): 0 SKUs
  Qty growing blocked (high DOH > 30): 0 SKUs

ENFORCING COMMERCIAL MINIMUM PRICE
  SKUs with commercial min: 0
  Growing SKUs with commercial min applied: 0
  SKUs only needing commercial min: 0
  Total commercial min enforcements: 0

ACTION SUMMARY

Price Actions:
price_action
none                      79
rets_growing               8
wac_increase               3
qty_growing_price_step     1

Cart Rule Actions:
cart_rule_action
none           89
qty_growing     2

SKUs with new price: 12
  Avg price change: 2.3%

SKUs with new cart rule: 2
  Avg cart rule change: -65.4%

SAMPLE

,sku,current_price,new_price,current_cart_rule,new_cart_rule,price_action,cart_rule_action,action_reason
5,مناديل سمايل سحب 550 منديل,247.50,257.614722,12,NaN,wac_increase,none,wac_increase
8,زيت ثمرات خليط - 700 مل,575.25,NaN,8,5.0,none,qty_growing,growing_performance
21,شويبس جولد اناناس - 1.75 لتر,204.25,205.635259,73,NaN,rets_growing,none,growing_performance
23,مناديل سمايل سحب 550 منديل,248.00,257.614722,5,NaN,wac_increase,none,wac_increase
26,ستينج فراولة مينى تربو - 250 مل,111.25,112.500000,11,NaN,rets_growing,none,growing_performance
27,فلامنكو بطعم السوداني الشوفان 10 جنية,91.00,92.250000,10,NaN,rets_growing,none,growing_performance
28,مسحوق باهى يدوى لافندر 12 كيس- 135 جم,90.25,93.750000,13,NaN,rets_growing,none,growing_performance
34,مسحوق باهي يدوي لافندر - 60 جم,90.00,91.183271,16,NaN,rets_growing,none,growing_performance
47,مسحوق غسيل فل ازرق يدوى - 2.5 كجم,54.25,55.720436,27,NaN,rets_growing,none,growing_performance
49,مناديل سمايل سحب 550 منديل,247.50,257.614722,12,NaN,wac_increase,none,wac_increase


In [14]:
# =============================================================================
# PUSH CONFIGURATION
# =============================================================================
# Mode: 'testing' = prepare files only, 'live' = actually upload to API
PUSH_MODE = 'live'  # Change to 'live' when ready to push

print(f"\n{'='*60}")
print(f"PUSH MODE: {PUSH_MODE.upper()}")
print(f"{'='*60}")
if PUSH_MODE == 'testing':
    print("⚠️ Testing mode - files will be prepared but NOT uploaded")
else:
    print("🚀 Live mode - files will be uploaded to MaxAB API")



PUSH MODE: LIVE
🚀 Live mode - files will be uploaded to MaxAB API


In [15]:
# Save df_action state before any manipulation for Slack upload later
temp_df_for_slack = df_action.copy()
print(f"✅ Saved {len(temp_df_for_slack)} rows for Slack upload")

✅ Saved 91 rows for Slack upload


In [16]:
# =============================================================================
# IMPORT PUSH HANDLERS
# =============================================================================
%run push_cart_rules_handler.ipynb
%run push_prices_handler.ipynb

print("✅ Push handlers loaded")


Push Cart Rules Handler loaded at 2026-02-09 18:04:31 Cairo time
✓ API credentials loaded successfully


Push Prices Handler loaded at 2026-02-09 18:04:31 Cairo time
✓ API credentials loaded successfully


✓ Google Sheets client initialized
✅ Push handlers loaded


In [17]:
# =============================================================================
# PREPARE DATA FOR PUSH
# =============================================================================
# Get packing units for push handlers
pus = get_packing_units()
print(f"✅ Loaded {len(pus)} packing units")

# Prepare df_action with required columns for push functions
# Prices need: product_id, sku, new_price, warehouse_id, cohort_id, stocks, current_price
# Cart rules need: product_id, sku, new_cart_rule, warehouse_id, cohort_id, stocks, current_cart_rule

# Rename stocks column if needed
if 'current_stocks' in df_action.columns and 'stocks' not in df_action.columns:
    df_action['stocks'] = df_action['current_stocks']

# Summary of what will be pushed
price_changes = df_action[df_action['new_price'].notna() & (df_action['new_price'] != df_action['current_price'])]
cart_changes = df_action[df_action['new_cart_rule'].notna() & (df_action['new_cart_rule'] != df_action['current_cart_rule'])]

print(f"\n📊 Push Summary:")
print(f"  Price changes: {len(price_changes)} SKUs")
print(f"  Cart rule changes: {len(cart_changes)} SKUs")
print(f"  Mode: {PUSH_MODE}")


Fetching packing_units ...


  Loaded 35126 records
✅ Loaded 35126 packing units

📊 Push Summary:
  Price changes: 12 SKUs
  Cart rule changes: 2 SKUs
  Mode: live


In [18]:
# =============================================================================
# STEP 1: PUSH CART RULES
# =============================================================================
# Push cart rules first - if any cohorts fail, we'll skip them for prices too
print(f"\n{'='*60}")
print("STEP 1: PUSH CART RULES")
print(f"{'='*60}")

cart_result = push_cart_rules(df_action, pus, source_module='module_4', mode=PUSH_MODE)

# Track failed cohorts to skip in price push
failed_cohorts = cart_result.get('failed_cohorts', [])

print(f"\n📊 Cart Rules Push Result:")
print(f"  Total received: {cart_result['total_received']}")
print(f"  Cart rule changes: {cart_result['cart_rule_changes']}")
print(f"  Pushed: {cart_result['pushed']}")
print(f"  Skipped: {cart_result['skipped']}")
print(f"  Failed: {cart_result['failed']}")
print(f"  Mode: {cart_result['mode']}")

if failed_cohorts:
    print(f"  ⚠️ Failed cohorts: {failed_cohorts}")



STEP 1: PUSH CART RULES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API

PUSH CART RULES - Source: module_4
Total received: 91
Cart rule changes to push: 2
Skipped (no change): 89

Cart rule changes summary:
  Increases: 0
  Decreases: 2

📋 Prepared 2 packing unit cart rules

Processing cohort: 702
  Saved: uploads/module_4_cart_rules_702.xlsx (1 rows)


  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 249.50it/s]

  Uploading...


...

  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 258.13it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 2
Total failed: 0

📊 Cart Rules Push Result:
  Total received: 91
  Cart rule changes: 2
  Pushed: 2
  Skipped: 89
  Failed: 0
  Mode: live


In [19]:
# =============================================================================
# STEP 2: PUSH PRICES (skip failed cohorts from cart rules)
# =============================================================================
print(f"\n{'='*60}")
print("STEP 2: PUSH PRICES")
print(f"{'='*60}")

# Push prices, skipping any cohorts that failed during cart rules push
push_result = push_prices(df_action, pus, source_module='module_4', mode=PUSH_MODE, skip_cohorts=failed_cohorts)

print(f"\n📊 Prices Push Result:")
print(f"  Total received: {push_result['total_received']}")
print(f"  Price changes: {push_result['price_changes']}")
print(f"  Pushed: {push_result['pushed']}")
print(f"  Skipped: {push_result['skipped']}")
print(f"  Failed: {push_result['failed']}")
print(f"  Mode: {push_result['mode']}")

if push_result.get('skipped_cohorts'):
    print(f"  ⚠️ Skipped cohorts: {push_result['skipped_cohorts']}")



STEP 2: PUSH PRICES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API
Loading disable_pu_visibility from Google Sheets...


  ✓ Loaded 88 products to disable min PU visibility

PUSH PRICES - Source: module_4
Total received: 91
Price changes to push: 12
Skipped (no change): 79

Price changes summary:
  Increases: 12
  Decreases: 0

📋 Prepared 12 packing unit prices

Processing cohort: 703
  Saved: uploads/module_4_703.xlsx (1 rows)


  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 223.28it/s]

...

  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 234.54it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 12
Total failed: 0

📊 Prices Push Result:
  Total received: 91
  Price changes: 12
  Pushed: 12
  Skipped: 79
  Failed: 0
  Mode: live


In [20]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("MODULE 4 - HOURLY UPDATES COMPLETE")
print(f"{'='*60}")
print(f"\n📅 Timestamp: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Mode: {PUSH_MODE.upper()}")

print(f"\n📊 ACTIONS TAKEN:")
print(f"  Total SKUs analyzed: {len(df)}")
print(f"  SKUs requiring action: {len(df_action)}")

# Price actions breakdown
print(f"\n💰 PRICE ACTIONS:")
if 'price_action' in df_action.columns:
    price_actions = df_action[df_action['price_action'] != 'none']
    print(f"  Total price changes: {len(price_actions)}")
    for action_type in df_action['price_action'].unique():
        if action_type != 'none':
            count = len(df_action[df_action['price_action'] == action_type])
            print(f"    - {action_type}: {count}")
else:
    print(f"  Total price changes: 0")

# Cart rule actions breakdown  
print(f"\n🛒 CART RULE ACTIONS:")
if 'cart_rule_action' in df_action.columns:
    cart_actions = df_action[df_action['cart_rule_action'] != 'none']
    print(f"  Total cart rule changes: {len(cart_actions)}")
    for action_type in df_action['cart_rule_action'].unique():
        if action_type != 'none':
            count = len(df_action[df_action['cart_rule_action'] == action_type])
            print(f"    - {action_type}: {count}")
else:
    print(f"  Total cart rule changes: 0")

# Push results
print(f"\n📤 PUSH RESULTS:")
print(f"  Cart Rules - Pushed: {cart_result.get('pushed', 0)}, Failed: {cart_result.get('failed', 0)}")
print(f"  Prices - Pushed: {push_result.get('pushed', 0)}, Failed: {push_result.get('failed', 0)}")

if PUSH_MODE == 'testing':
    print(f"\n⚠️ TESTING MODE - No changes were actually pushed to production!")
    print(f"   Change PUSH_MODE to 'live' to execute actual pushes.")
else:
    print(f"\n✅ LIVE MODE - Changes have been pushed to production!")



MODULE 4 - HOURLY UPDATES COMPLETE

📅 Timestamp: 2026-02-09 18:05:04
🔧 Mode: LIVE

📊 ACTIONS TAKEN:
  Total SKUs analyzed: 28084
  SKUs requiring action: 91

💰 PRICE ACTIONS:
  Total price changes: 12
    - wac_increase: 3
    - rets_growing: 8
    - qty_growing_price_step: 1

🛒 CART RULE ACTIONS:
  Total cart rule changes: 2
    - qty_growing: 2

📤 PUSH RESULTS:
  Cart Rules - Pushed: 2, Failed: 0
  Prices - Pushed: 12, Failed: 0

✅ LIVE MODE - Changes have been pushed to production!


In [21]:
# =============================================================================
# SAMPLE OUTPUT - Current Status
# =============================================================================
# Show sample of data with all calculated fields

sample_cols = [
    'warehouse_id', 'product_id', 'sku','wac1','wac_p','new_wac',
    # P80/P70 benchmarks and std
    'p80_daily_240d', 'std_daily_240d', 'p70_daily_retailers_240d', 'std_daily_retailers_240d',
    # Current cart rule
    'current_cart_rule',
    # UTH performance (with std thresholds)
    'uth_qty', 'uth_qty_target', 'uth_qty_std', 'uth_qty_status',
    'uth_retailers', 'uth_rets_target', 'uth_rets_std', 'uth_rets_status',
    # Last hour performance (with std thresholds)
    'last_hour_qty', 'last_hour_qty_target', 'last_hour_qty_std', 'last_hour_qty_status',
    'last_hour_retailers', 'last_hour_rets_target', 'last_hour_rets_std', 'last_hour_rets_status'
]

# Filter to columns that exist
sample_cols = [c for c in sample_cols if c in df.columns]

print(f"\n{'='*60}")
print("SAMPLE DATA (First 10 rows with UTH > 0)")
print(f"{'='*60}")
sample = df[df['uth_qty'] > 0][sample_cols].head(10)
display(sample)



SAMPLE DATA (First 10 rows with UTH > 0)


,warehouse_id,product_id,sku,wac1,wac_p,new_wac,p80_daily_240d,std_daily_240d,p70_daily_retailers_240d,std_daily_retailers_240d,...,uth_rets_std,uth_rets_status,last_hour_qty,last_hour_qty_target,last_hour_qty_std,last_hour_qty_status,last_hour_retailers,last_hour_rets_target,last_hour_rets_std,last_hour_rets_status
1,170,9787,عصير جهينة بيور كوكتيل - 235 مل,404.643000,385.422457,385.422457,5.0,3.349062,5.00,2.112791,...,1.300711,on_track,0.0,0.329376,0.220620,dropping,0.0,0.328149,0.138662,dropping
5,401,10721,حفاضات بيبى جوى كلوت مضغوط جامبو مقاس 6 - 46 ح...,301.821156,273.778805,273.781281,5.0,1.370968,5.00,0.399351,...,0.218051,dropping,0.0,0.263422,0.072229,dropping,0.0,0.282100,0.022531,dropping
7,501,1667,نسكافيه 3*1 بندق 24 ظرف - 18 جم,127.766723,127.766723,127.757756,5.0,1.621567,5.00,0.984524,...,0.601647,dropping,0.0,0.314258,0.101918,dropping,0.0,0.330699,0.065116,dropping
8,797,24244,كيك كمارا تورنيدو كريمة الفانيليا و مربى الفرا...,22.810000,22.125700,22.125700,5.0,1.224745,5.00,0.816497,...,0.500301,dropping,0.0,0.341139,0.083562,dropping,0.0,0.320742,0.052377,dropping
11,797,12505,بريكس مزيل بقع -فائق الفعاليه للابيض والالوان ...,426.244322,404.932295,404.932295,5.0,0.315301,5.00,0.315301,...,0.186728,dropping,0.0,0.367422,0.023170,dropping,0.0,0.333082,0.021004,dropping
12,703,11587,اوكسى سائل اطباق ليمون أصفر ظرف 35 جرام,70.141744,68.646122,68.646122,5.0,3.671371,5.00,1.055368,...,0.633002,on_track,0.0,0.313716,0.230354,dropping,0.0,0.356977,0.075348,dropping
14,501,130,لبن بخيره - 500 مل,506.389835,481.064791,481.064791,32.4,24.236276,15.00,5.637326,...,3.541528,on_track,26.0,2.076732,1.553465,growing,3.0,0.981493,0.368866,growing
18,962,11952,اوكسي يدوي - 175 جم,144.583101,140.005149,140.005149,15.0,10.142861,6.00,3.867125,...,2.421938,growing,1.0,1.016984,0.687675,on_track,1.0,0.410699,0.264704,on_track
19,8,4222,بن عبد المعبود سادة وسط - 100 جم,558.807945,537.487900,537.487900,5.0,1.051010,5.00,0.883075,...,0.537590,dropping,0.0,0.355739,0.074777,dropping,0.0,0.340605,0.060156,dropping
21,236,13283,فلامنكو ذرة فشار بطعم الطماطم - 10 جنية,85.500000,85.500000,85.500000,14.6,5.203354,7.51,3.015535,...,1.858317,on_track,0.0,0.935484,0.333401,dropping,0.0,0.469216,0.188407,dropping


In [22]:
# =============================================================================
# UPLOAD RESULTS TO SNOWFLAKE AND SEND SLACK NOTIFICATION
# =============================================================================

from common_functions import upload_dataframe_to_snowflake, send_text_slack, send_file_slack

SLACK_CHANNEL_ID = 'C0AAWK97Z3Q'

# Add created_at as TIMESTAMP (module runs hourly)
df_action['created_at'] = datetime.now(CAIRO_TZ).replace(second=0, microsecond=0)
# Drop columns not in database schema (internal columns used for logic only)
df_action = df_action.drop(columns=['commercial_min_price', 'doh', 'qty_price_step_count'], errors='ignore')
# Upload to Snowflake
print(f"\n{'='*60}")
print("UPLOADING RESULTS TO SNOWFLAKE")
print(f"{'='*60}")

upload_status = upload_dataframe_to_snowflake(
    "Egypt", 
    df_action, 
    "MATERIALIZED_VIEWS", 
    "pricing_hourly_push", 
    "append", 
    auto_create_table=True, 
    conn=None
)

# Prepare status variables
prices_pushed = push_result.get('pushed', 0) if 'push_result' in dir() else 0
prices_failed = push_result.get('failed', 0) if 'push_result' in dir() else 0
cart_rules_pushed = cart_result.get('pushed', 0) if 'cart_result' in dir() else 0
cart_rules_failed = cart_result.get('failed', 0) if 'cart_result' in dir() else 0

# Count price and cart rule actions
price_changes = len(df_action[df_action['price_action'] != 'none']) if 'price_action' in df_action.columns else 0
cart_changes = len(df_action[df_action['cart_rule_action'] != 'none']) if 'cart_rule_action' in df_action.columns else 0

if upload_status:
    slack_message = f"""✅ *Module 4 - Hourly Updates Completed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Hour: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
🔧 Mode: {PUSH_MODE.upper()}

📊 *Results:*
• Total SKUs analyzed: {len(df):,}
• SKUs requiring action: {len(df_action):,}
• Price changes: {price_changes:,}
• Cart rule changes: {cart_changes:,}

📤 *Push Status:*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed

🗄️ Results uploaded to: MATERIALIZED_VIEWS.pricing_hourly_push"""
    
    send_text_slack('new-pricing-logic', slack_message)
    print("✅ Slack notification sent!")
    
    # Send review file to Slack after the text message (using saved copy before manipulation)
    send_file_slack(
        temp_df_for_slack, 
        f'📎 Module 4 Review: {len(temp_df_for_slack)} SKUs processed', 
        SLACK_CHANNEL_ID,
        filename=f'module4_hourly_{datetime.now(CAIRO_TZ).strftime("%Y%m%d_%H%M")}.xlsx'
    )
    print("✅ Review file sent to Slack")
    
    print(f"✅ {len(df_action)} records uploaded to Snowflake")
else:
    error_message = f"""❌ *Module 4 - Hourly Updates Failed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Hour: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
⚠️ Upload to Snowflake failed - please check logs

📤 *Push Status (before upload failure):*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed"""
    
    send_text_slack('new-pricing-logic', error_message)
    print("❌ Error notification sent to Slack!")
    
    # Still send review file even on error for debugging (using saved copy before manipulation)
    send_file_slack(
        temp_df_for_slack, 
        f'⚠️ Module 4 ERROR Review: {len(temp_df_for_slack)} SKUs', 
        SLACK_CHANNEL_ID,
        filename=f'module4_hourly_ERROR_{datetime.now(CAIRO_TZ).strftime("%Y%m%d_%H%M")}.xlsx'
    )
    print("✅ Error review file sent to Slack")



UPLOADING RESULTS TO SNOWFLAKE


/home/ec2-user/service_account_key.json


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://docs.slack.dev/tools/python-slack-sdk/v3-migration/
  warnings.warn(message)


/home/ec2-user/service_account_key.json


Message Sent
✅ Slack notification sent!


/home/ec2-user/service_account_key.json


File module4_hourly_20260209_1805.xlsx sent to Slack
✅ Review file sent to Slack
✅ 91 records uploaded to Snowflake
